In [9]:
import pandas as pd
import requests
from xml.etree import ElementTree as ET
from io import StringIO
from bs4 import BeautifulSoup

The information in this document takes an XML target for an NPORT form and returns information about it.

<hr>
The following block of code imports the pickle file from 01 then selects a sample. The sample is selected randomly, but by setting the seed, the sample will the be same each time this block of code is run.

In [2]:
df = pd.read_pickle('Investment Company Series and Class Information.p')
sample = df.sample(random_state=35)
sample

,reporting_file_number,cik,entity_name,entity_org_type,series_id,series_name,class_id,class_name,class_ticker,address_1,city,zip_code,state,address_2
15962,811-04347,0000772129,GMO TRUST,30,S000005487,GMO Global Asset Allocation Fund,C000213377,Class I,[NULL],40 ROWES WHARF,BOSTON,02110,MA,[NULL]


I manually found an example N-PORT filing for this company. This example is the [NPORT monthly filing from November 30, 2019](https://www.sec.gov/Archives/edgar/data/772129/000175272420015077/0001752724-20-015077-index.htm). This block of code imports that into an ElementTree

In [21]:
link = 'https://www.sec.gov/Archives/edgar/data/772129/000175272420015077/primary_doc.xml'

r = requests.get(link)
assert(r), r
xml = ET.fromstring(r.text)

# Remove namespace tags
xml = ET.iterparse(StringIO(r.text))
for _, el in xml:
    _, _, el.tag = el.tag.rpartition('}') # strip ns
xml = xml.root

In [19]:
list(xml.find('formData'))

[<Element 'genInfo' at 0x000002374144E728>,
 <Element 'fundInfo' at 0x000002374144BD18>,
 <Element 'invstOrSecs' at 0x000002374147A2C8>,
 <Element 'explntrNotes' at 0x00000237414CCA98>,
 <Element 'signature' at 0x00000237414CCC28>]

In this block of code, I build the portion of the function that accepts the xml ElementTree then extracts information about the investments --`invstOrSecs`--they hold.

In [22]:

secs = xml.find('formData').find('invstOrSecs')
all_secs = []
for s in secs:
    t_dict = {}
    for e in s:
        t_dict[e.tag] = e.text 
    all_secs += [t_dict]
res = pd.DataFrame(all_secs)
        
# Extra data cleaning to convert strings to numbers
num_cols = ['valUSD','balance','pctVal']

for c in num_cols:
    res[c] = res[c].astype(float)

res.head()

,name,lei,title,cusip,identifiers,balance,units,currencyConditional,valUSD,pctVal,payoffProfile,assetCat,issuerCat,invCountry,isRestrictedSec,fairValLevel,securityLending,curCd,issuerConditional,derivativeInfo
0,Ebara Corp,353800UV8U3SOMCDTI49,Ebara Corp,000000000,\n,35900.0,NS,NaN,1080297.91,0.915719,Long,EC,CORP,JP,N,2,\n,NaN,NaN,NaN
1,Bakkafrost P/F,2138007LH7OP4V112978,Bakkafrost P/F,000000000,\n,704.0,NS,NaN,47197.02,0.040007,Long,EC,CORP,FO,N,2,\n,NaN,NaN,NaN
2,METAWATER Co Ltd,N/A,METAWATER Co Ltd,000000000,\n,4300.0,NS,NaN,156886.67,0.132986,Long,EC,CORP,JP,N,2,\n,NaN,NaN,NaN
3,Tongaat Hulett Ltd,N/A,Tongaat Hulett Ltd,000000000,\n,19722.0,NS,NaN,8891.42,0.007537,Long,EC,CORP,ZA,N,3,\n,NaN,NaN,NaN
4,Exelon Corp,3SOUA6IRML7435B56G12,Exelon Corp,30161N101,\n,26500.0,NS,NaN,1176600.00,0.997350,Long,EC,CORP,US,N,1,\n,USD,NaN,NaN


In [47]:
res['valUSD'].sum()

117928094.39999999